# Play Your Dogs Right game

This game involves the player betting on whether the winning trap number in a sequence of six races is 'Higher or Lower' than the previous race, based on a starting number of 3.5.

The bet requires the player to determine whether, for each race, the winning trap number was higher or lower than the previous trap number. An example of a selection might be “HLHLHL”. The first result is evaluated assuming that the previous trap number was 3.5 (so it must be either higher or lower—equal is not possible when comparing the first trap).

The player is paid a dividend according to the length of the run they predict. So, if the race results are 1, 2, 3, 4, 5, 6 and the player selects, LHHLHH, then they match a run of three.

If the player fails to correctly predict whether the _first_ trap is $$3.5 or $$3.5, then an attempt is made to match the remainder of the traps. The player is given a “pass” for an _inital_ mistake, but it doesn’t count to the matched sequence length. For example,  if the race results are 1, 2, 3, 4, 5, 6, and the player selects HHHLHH, then they match a run of two. Note that the pass only applies to the _first_ trap.

There is a variation, allowing the player to bet on a ‘Double Chance’. The Double Chance market involves the player paying a double stake to buy insurance against making a single mistake in their prediction for any of races 2, 3, 4 and 5. If the player fails correctly to predict Higher or Lower in any of these races, the Double Chance insurance kicks in and ensures a run continues as though the wrong prediction didn’t happen. The wrong prediction does not count towards the run length.

Double Chance insurance can only be applied once per game, and is only applied to races 2 thru’ 5. If the insurance element of the stake is not applied the stake is not returned.

The win market dividends are:

| Run | Dividend |
| --: | -------: |
| 6   | 10       |
| 5   | 5        |
| 4   | 3        |
| 3   | 1        |

The Double Chance dividends are:

| Run | Dividend |
| --: | -------: |
| 6   | 12       |
| 5   | 7        |
| 4   | 5        |
| 3   | 2        |

## Configuration

In [ ]:
import random
from itertools import product
from pprint import pprint as pp

## Potential outcomes

Generate all 46656 possible outcomes.

In [ ]:
traps = list(range(1, 7))
potential_outcomes = list(product(traps, traps, traps, traps, traps, traps))

assert len(potential_outcomes) == 6**6

pp(potential_outcomes[:5])
pp(potential_outcomes[-5:])
pp(random.sample(potential_outcomes, 5))

## Settlement

In [ ]:
def get_run_length(outcome, selection, insured=False):
    previous_trap = 3.5
    insurance_available = insured
    run_length = 0

    for index, (trap, choice) in enumerate(zip(outcome, selection)):
        correct_choice = (choice == "L" and trap < previous_trap) or (
            choice == "H" and trap > previous_trap
        )

        previous_trap = trap

        if correct_choice:
            run_length += 1

            continue

        if index == 0:
            continue

        if insurance_available:
            insurance_available = False

            continue

        break

    return run_length

In [ ]:
assert get_run_length((1, 2, 3, 4, 5, 6), "LHLHLL", insured=True) == 3
assert get_run_length((1, 2, 3, 4, 5, 6), "HHHLHH", insured=True) == 4
assert get_run_length((1, 2, 3, 4, 5, 6), "HLHHLH", insured=True) == 2

## Simulation

In [ ]:
def simulate(selection, insured=False):
    assert set(selection) == {"H", "L"}

    run_length_frequencies = {}

    for outcome in potential_outcomes:
        run_length = get_run_length(outcome, selection, insured)

        run_length_frequencies[run_length] = (
            run_length_frequencies.get(run_length, 0) + 1
        )

    assert sum(run_length_frequencies.values()) == 6**6

    run_length_probabilities = {
        run_length: run_length_frequencies.get(run_length, 0) / 6**6
        for run_length in range(0, 7)
    }

    assert abs(1 - sum(run_length_probabilities.values())) < 1e-9

    return run_length_probabilities

## Optimal strategy

In [ ]:
optimal_selection = "HLHLHL"

## Win market

In [ ]:
win_probabilities = simulate(optimal_selection)

win_probabilities

## Insured market

In [ ]:
insured_probabilities = simulate(optimal_selection, insured=True)

insured_probabilities

## Expected RTP

The prices offered on the various run lengths are:

The expected RTP for a win bet is:

In [ ]:
win_dividends = {6: 10, 5: 5, 4: 3, 3: 1}

In [ ]:
sum(
    win_probabilities.get(run_length, 0) * win_dividends.get(run_length, 0)
    for run_length in range(0, 7)
)

The expected RTP for a Double Chance bet is:

In [ ]:
double_chance_dividends = {6: 12, 5: 7, 4: 5, 3: 2}

In [ ]:
sum(
    insured_probabilities.get(run_length, 0) * double_chance_dividends.get(run_length, 0)
    for run_length in range(0, 7)
) / 2